In [1]:
%pip install -q git+https://github.com/tensorflow/docs

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorflow.keras.utils import get_file
import tensorflow as tf
from tensorflow.python.data.experimental import CsvDataset

data_package = get_file('HIGGS.csv.gz', 'http://mlphysics.ics.uci.edu/data/higgs/HIGGS.csv.gz')

FEATURES = 28
dataset = CsvDataset(data_package, [float(), ] * (FEATURES + 1), compression_type='GZIP')

In [3]:
def unpack_row(*row):
    label = row[0]
    features = tf.stack(row[1:], 1)
    return features, label
processed_dataset = dataset.batch(10_000).map(unpack_row).unbatch()

BATCH_SIZE = 500
TRAIN_SIZE = 10_000
VALIDATION_SIZE = 1_000
BUFFER_SIZE = 10_000
STEPS_PER_EPOCH = TRAIN_SIZE // BATCH_SIZE

validation_dataset = processed_dataset.take(VALIDATION_SIZE).cache()
validation_dataset = validation_dataset.batch(BATCH_SIZE)

train_dataset = processed_dataset.skip(VALIDATION_SIZE).take(TRAIN_SIZE).cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

In [4]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import InverseTimeDecay
from tensorflow_docs.modeling import EpochDots
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryCrossentropy as BinaryCrossentropyMetrics
import pathlib
import shutil
import tempfile

logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)
%load_ext tensorboard

def get_optimizer():
    learning_rate_schedule = InverseTimeDecay(0.001, decay_steps=STEPS_PER_EPOCH * 1000, decay_rate=1, staircase=False)
    return Adam(learning_rate_schedule)

def get_callbacks(name):
    return [
        EpochDots(),
        EarlyStopping(monitor='val_binary_crossentropy', patience=200),
        TensorBoard(logdir / name)
    ]

def compile_and_fit(model: Sequential, name, optimizer=None, max_epochs=1_000):
    optimizer = optimizer or get_optimizer()
    model.compile(
        optimizer=optimizer, 
        loss=BinaryCrossentropy(from_logits=True),
        metrics=[
            BinaryCrossentropyMetrics(from_logits=True),
            'accuracy'
        ])
    model.summary()
    return model.fit(
        train_dataset, steps_per_epoch=STEPS_PER_EPOCH, epochs=max_epochs, 
        validation_data=validation_dataset, callbacks=get_callbacks(name), verbose=0)

size_histories = {}
regularizer_histories = {}

In [5]:
from tensorflow.keras.layers import Dense

shutil.rmtree(logdir / 'sizes/tiny', ignore_errors=True)
tiny_model = Sequential([
    Dense(16, activation='elu', input_shape=(FEATURES,)),
    Dense(1)
])

size_histories['tiny'] = compile_and_fit(tiny_model, 'sizes/tiny')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                464       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________

Epoch: 0, accuracy:0.4786,  binary_crossentropy:1.1478,  loss:1.1478,  val_accuracy:0.4530,  val_binary_crossentropy:0.9724,  val_loss:0.9724,  
....................................................................................................
Epoch: 100, accuracy:0.5850,  binary_crossentropy:0.6298,  loss:0.6298,  val_accuracy:0.5910,  val_binary_crossentropy:0.6301,  val_loss:0.6301,  
............................................................

In [6]:
shutil.rmtree(logdir / 'sizes/small', ignore_errors=True)

small_model = Sequential([
    Dense(16, activation='elu', input_shape=(FEATURES,)),
    Dense(16, activation='elu'),
    Dense(1)
])
size_histories['small'] = compile_and_fit(small_model, 'sizes/small')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 16)                464       
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 1)                 17        
                                                                 
Total params: 753
Trainable params: 753
Non-trainable params: 0
_________________________________________________________________

Epoch: 0, accuracy:0.5081,  binary_crossentropy:0.8286,  loss:0.8286,  val_accuracy:0.4900,  val_binary_crossentropy:0.7410,  val_loss:0.7410,  
....................................................................................................
Epoch: 100, accuracy:0.6169,  binary_crossentropy:0.6182,  loss:0.6182,  

In [7]:
shutil.rmtree(logdir / 'sizes/medium', ignore_errors=True)

medium_model = Sequential([
    Dense(64, activation='elu', input_shape=(FEATURES,)),
    Dense(64, activation='elu'),
    Dense(64, activation='elu'),
    Dense(1)
])
size_histories['medium'] = compile_and_fit(medium_model, 'sizes/medium')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                1856      
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 10,241
Trainable params: 10,241
Non-trainable params: 0
_________________________________________________________________

Epoch: 0, accuracy:0.5020,  binary_crossentropy:0.6978,  loss:0.6978,  val_accuracy:0.4720,  val_binary_crossentropy:0.6766,  val_loss:0.6766,  
....................................

In [8]:
shutil.rmtree(logdir / 'sizes/large', ignore_errors=True)

large_model = Sequential([
    Dense(512, activation='elu', input_shape=(FEATURES,)),
    Dense(512, activation='elu'),
    Dense(512, activation='elu'),
    Dense(512, activation='elu'),
    Dense(1)
])
size_histories['large'] = compile_and_fit(large_model, 'sizes/large')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 512)               14848     
                                                                 
 dense_10 (Dense)            (None, 512)               262656    
                                                                 
 dense_11 (Dense)            (None, 512)               262656    
                                                                 
 dense_12 (Dense)            (None, 512)               262656    
                                                                 
 dense_13 (Dense)            (None, 1)                 513       
                                                                 
Total params: 803,329
Trainable params: 803,329
Non-trainable params: 0
_________________________________________________________________

Epoch: 0, accuracy:0.4971,  binary_crossentropy

In [10]:
%reload_ext tensorboard
%tensorboard --logdir {logdir}/sizes

Reusing TensorBoard on port 6006 (pid 98151), started 0:00:17 ago. (Use '!kill 98151' to kill it.)

In [11]:
shutil.rmtree(logdir / 'regularizers', ignore_errors=True)
shutil.copytree(logdir / 'sizes/tiny', logdir / 'regularizers/tiny')
regularizer_histories['tiny'] = size_histories['tiny']

In [12]:
from tensorflow.keras import regularizers

shutil.rmtree(logdir / 'regularizers/l2', ignore_errors=True)

l2_model = Sequential([
    Dense(512, activation='elu', kernel_regularizer=regularizers.l2(0.001), input_shape=(FEATURES,)),
    Dense(512, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
    Dense(512, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
    Dense(512, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
    Dense(1)
])

regularizer_histories['l2'] = compile_and_fit(l2_model, 'regularizers/l2')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               14848     
                                                                 
 dense_15 (Dense)            (None, 512)               262656    
                                                                 
 dense_16 (Dense)            (None, 512)               262656    
                                                                 
 dense_17 (Dense)            (None, 512)               262656    
                                                                 
 dense_18 (Dense)            (None, 1)                 513       
                                                                 
Total params: 803,329
Trainable params: 803,329
Non-trainable params: 0
_________________________________________________________________

Epoch: 0, accuracy:0.5051,  binary_crossentropy

In [13]:
from tensorflow.keras.layers import Dropout

shutil.rmtree(logdir / 'regularizers/dropout', ignore_errors=True)

l2_model = Sequential([
    Dense(512, activation='elu', input_shape=(FEATURES,)),
    Dropout(0.5),
    Dense(512, activation='elu'), 
    Dropout(0.5),
    Dense(512, activation='elu'),
    Dropout(0.5),
    Dense(512, activation='elu'),
    Dropout(0.5),
    Dense(1)
])

regularizer_histories['dropout'] = compile_and_fit(l2_model, 'regularizers/dropout')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 512)               14848     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_20 (Dense)            (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 512)               262656    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_22 (Dense)            (None, 512)              

In [20]:
shutil.rmtree(logdir / 'regularizers/combined', ignore_errors=True)

combined_model = Sequential([
    Dense(512, activation='elu', kernel_regularizer=regularizers.l2(0.0001), input_shape=(FEATURES,)),
    Dropout(0.5),
    Dense(512, activation='elu', kernel_regularizer=regularizers.l2(0.0001)), 
    Dropout(0.5),
    Dense(512, activation='elu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.5),
    Dense(512, activation='elu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.5),
    Dense(1)
])

regularizer_histories['combined'] = compile_and_fit(combined_model, 'regularizers/combined')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 512)               14848     
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_35 (Dense)            (None, 512)               262656    
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                                 
 dense_36 (Dense)            (None, 512)               262656    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 dense_37 (Dense)            (None, 512)              

In [21]:
%tensorboard --logdir {logdir}/regularizers

Reusing TensorBoard on port 6007 (pid 182630), started 7:42:20 ago. (Use '!kill 182630' to kill it.)